In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne
import os
import math

In [ ]:
def find_min_num_of_file(dir):
    print(f'========= The process of finding the optimal number of images for each stage has been started. =========')
    print(f'========= During the process files of the supplied directory will be analysed. =========\n')

    all_files_in_dir = os.listdir(dir)
    stages = [0, 0, 0, 0, 0, 0]

    all_psg_files = []
    all_hyp_files = []
    checked = 0
    annotation_desc_2_event_id = {
        "Sleep stage W": 1,
        "Sleep stage 1": 2,
        "Sleep stage 2": 3,
        "Sleep stage 3": 4,
        "Sleep stage 4": 5,
        "Sleep stage R": 6,
    }

    for file in all_files_in_dir:
        if file[0] == '.':    # for ignoring hidden files in the directory
            continue
        parts = file.split("-")
        if parts[1] == "PSG.edf":
            all_psg_files.append(file)
        elif parts[1] == "Hypnogram.edf":
            all_hyp_files.append(file)

    for file in all_psg_files:
        
        current_psg_file = dir + "/" + file

        hyp_file = file.split("-")[0][:-2] + "*" + "-Hypnogram.edf"
        possible_hyp_file = fnmatch.filter(all_hyp_files, hyp_file)

        if possible_hyp_file:

            current_hyp_file = dir + "/" + possible_hyp_file[0]

            data = mne.io.read_raw_edf(current_psg_file, stim_channel="Event marker", ifer_types=True, preload=True, verbose=False)

            annot_train = mne.read_anotations(current_hyp_file)

            annot_train.crop(annot_train[1]["onset"] - 30*60, annot_train[-2]["onset"] + 30*60)
            data.set_annotations(annot_train, emit_warning=False)


            events_train, _ = mne.events_from_annotations(
                data, event_id=annotation_desc_2_event_id, chunk_duration=30.0, verbose=False
            )

            for event in events_train:
                stages[event[2] - 1] += 1
            
            checked += 1

            print(f'========= Files have been analysed {checked}/{len(all_psg_files)} =========', end="\r")
        else:
            print(f'\n========= No such hypnogram file for {file} =========\n')

    print(f'\n========= Analysing has been done. =========\n')

    i = 0
    for stage in list(annotation_desc_2_event_id):
        print(f'========= Number of files for {stage} is {stage[i]} =========')
        i += 1

    print(f'========= Minimum number of files for each stage is {min(stages)} =========')

    return min(stages), max(stages)

In [3]:
def file_keeper(psg_file, hypnogram_file):
    data = mne.io.read_raw_edf(psg_file, stim_channel="Event marker", infer_types=True, preload=True)
    annotations = mne.read_annotations(hypnogram_file)
    return data, annotations

In [5]:
def all_sleep_stages(annotations):
    sleep_stages = []
    for i in annotations:
        if i.get('desription') in sleep_stages:
            pass
        else:
            sleep_stages.append(i.get('description'))
    return sleep_stages

In [1]:
def crop_set_annotations(data, annotations):
    annotations.crop(annotations[1]["onset"] - 30*60, annotations[-2]["onset"] + 30*60)
    data.set_annotatios(annotations, emit_warning=False)

    annotation_stage_id = {
        "Sleep stage W": 1,
        "Sleep stage 1": 2,
        "Sleep stage 2": 3,
        "Sleep stage 3": 4,
        "Sleep stage 4": 5,
        "Sleep stage R": 6,
    }

    events_train,_ = mne.events_from_annotations(
        data, evet_id=annotation_stage_id, chunk_duration=30.0
    )
    return events_train, annotation_stage_id